<a href="https://colab.research.google.com/github/Batman9698/Iris-classification-using-keras/blob/master/Transferlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import time
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [0]:
img_dir = '/content/drive/My Drive/Data/Cats and Dogs/data/Cat_Dog_data'

In [4]:
data_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])
train_images = ImageFolder(img_dir+'/train', transform = data_transforms)
test_images = ImageFolder(img_dir+'/test', transform = data_transforms)
trainLoader = DataLoader(train_images, batch_size = 256, shuffle = True, num_workers = 6)
testloader = DataLoader(test_images, batch_size = 256, shuffle = True, num_workers = 6)

device0 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device1 = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print("device: ",device0)
print("device: ",device1)

device:  cuda:0
device:  cuda:1


In [5]:
model = models.resnet18(pretrained = True)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
fc = nn.Sequential(nn.Linear(512, 256),
                   nn.ReLU(),
                   nn.Linear(256, 2),
                   nn.Softmax(dim=1))
for param in model.parameters():
    param.require_grad = False

model.fc = fc
model = model.to(device0)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.8)

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [0]:
def train(epochs):
    model.train()
    start = time.time()
    running_accuracy = 0
    running_loss = 0.0
    for epoch in range(epochs):
        for inputs, labels in trainLoader:
            inputs = inputs.to(device0)
            labels = labels.to(device0)

            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_accuracy += torch.sum(preds == labels.data)
            
        
        epoch_loss = running_loss / len(train_images)
        epoch_accuracy = running_accuracy.double() / len(test_images)

        print('epoch {}/{} \n loss: {} Accuracy: {}'.format(epoch, epochs-1, epoch_loss, epoch_accuracy))

In [8]:
train(15)

epoch 0/14 
 loss: 0.0015494808197021484 Accuracy: 8.5324
epoch 1/14 
 loss: 0.0028464826636844212 Accuracy: 17.4084
epoch 2/14 
 loss: 0.004120359224743313 Accuracy: 26.3216
epoch 3/14 
 loss: 0.0053809796147876315 Accuracy: 35.2612
epoch 4/14 
 loss: 0.006632777541213566 Accuracy: 44.2152
epoch 5/14 
 loss: 0.007879315519332885 Accuracy: 53.177600000000005
epoch 6/14 
 loss: 0.009121976415316264 Accuracy: 62.147200000000005
epoch 7/14 
 loss: 0.010360786100228628 Accuracy: 71.12440000000001
epoch 8/14 
 loss: 0.011597900569438935 Accuracy: 80.10640000000001
epoch 9/14 
 loss: 0.012832611940966712 Accuracy: 89.0908
epoch 10/14 
 loss: 0.014066413711176978 Accuracy: 98.078
epoch 11/14 
 loss: 0.01529931320614285 Accuracy: 107.0664
epoch 12/14 
 loss: 0.016531364284621344 Accuracy: 116.05560000000001
epoch 13/14 
 loss: 0.017762929989231957 Accuracy: 125.04480000000001
epoch 14/14 
 loss: 0.01899434433778127 Accuracy: 134.03480000000002
